In [2]:
import numpy as np
import scipy.io

In [3]:
#load everything
mat = scipy.io.loadmat('D:/Users/Ziqiu/OneDrive/Documents/University Docs/Stat441/A2/faces.mat')
training_data = np.concatenate((mat['train_faces'], mat['train_nonfaces']))/255
test_data = np.concatenate((mat['test_faces'], mat['test_nonfaces']))/255
y_training = np.concatenate((np.ones((2429,1)),np.zeros((2429,1))))
y_test = np.concatenate((np.ones((472,1)),np.zeros((472,1))))
#row-wise observations


In [4]:
#add a vector of 1s for beta_0 intercept
t = np.ones((944,1))
s = np.ones((4858,1))
test_data = np.concatenate((test_data,t),axis=1) 
training_data = np.concatenate((training_data, s), axis=1) 

In [5]:
#Pi and Score Vector
def pi(x, beta):
    return 1 - 1 / (1 + np.exp(x @ beta))

beta_old = np.zeros((362,1))
p = pi(training_data, beta_old)

def score(y, x, p):
    return x.transpose() @ (y - p)

In [6]:
#Information matrix

def information(x, p):
    diag = np.empty([0,1])
    for i in range(np.shape(p)[0]):
        diag = np.vstack((diag, p[i]*(1-p[i])))
    W = np.diag(diag.transpose()[0])
    return np.linalg.inv(x.transpose() @ W @ x)

information(training_data, p)

beta_new = beta_old + information(training_data, p) @ score(y_training, training_data, p)

In [7]:
#loop
for i in range(100):
    beta_old = beta_new
    p = pi(training_data, beta_old)
    beta_new = beta_old + information(training_data, p) @ score(y_training, training_data, p)

In [8]:
# Analyze Error Rates
E1 = training_data @ beta_new
error_training = 0
for i in range(np.shape(E1[0:2429])[0]):
    if E1[i] < 0:
        error_training += 1
for i in range(np.shape(E1[2429:4858])[0]):
    if E1[2429:4858][i] > 0:
        error_training +=1

In [9]:
training_error = error_training / 4858
# training error is 0.0146

In [10]:
E2 = test_data @ beta_new
error_test = 0
for i in range(np.shape(E2[0:472])[0]):
    if E1[i] < 0:
        error_test += 1
for i in range(np.shape(E2[472:944])[0]):
    if E1[472:944][i] > 0:
        error_test +=1

In [12]:
test_error = error_test/944
test_error

0.4915254237288136